<a href="https://colab.research.google.com/github/akshay4n/Quantitaive-Research/blob/main/quantitativeValueStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 14.1 MB/s 


In [2]:
#value investing -> investing in the stocks that are cheapest relative to common measures of business values(like earnings or assets)
#import
import numpy as np
import pandas as pd
import xlsxwriter
import math
import requests
from scipy import stats

In [3]:
path = '/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/sp_500_stocks.csv'
stocks = pd.read_csv(path)

In [4]:
%cd /content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/
from secrets_imp import IEX_CLOUD_API_TOKEN

/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500


In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 67207022,
 'calculationPrice': 'close',
 'change': -2.76,
 'changePercent': -0.0155,
 'close': 178.65,
 'closeSource': 'offilcia',
 'closeTime': 1707660438693,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 173.59,
 'delayedPriceTime': 1712674559027,
 'extendedChange': -0.77,
 'extendedChangePercent': -0.0046,
 'extendedPrice': 173.45,
 'extendedPriceTime': 1674415634660,
 'high': 181.15,
 'highSource': 'olCes',
 'highTime': 1678624280883,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 176.99,
 'iexCloseTime': 1722344145355,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 179.536,
 'iexOpenTime': 1686420706269,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1701828741489,
 'latestPrice': 171.62,
 'latestSource': 'Close',
 'latestTime': 'August 19, 2022',
 'latestUpdate': 1716798249242,
 'latestVolume': 71517452,
 'low': 176.71,
 'lowSourc

In [6]:
price = data['latestPrice']
pe_ratio = data['peRatio']

In [7]:
#batch api calls
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks['Ticker'][1:116],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker','Price', 'Price-to-Earnings Ratio','Number of Shares to Buy']

In [8]:
final_df = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAL,14.40,-5.1,N/A
1,AAP,210.27,23.34,N/A
2,AAPL,173.70,28.73,N/A
3,ABBV,142.64,20.58,N/A
4,ABC,152.64,18.02,N/A
...,...,...,...,...
110,COTY,7.80,43.76,N/A
111,CPB,52.66,17.11,N/A
112,CPRT,129.41,29.81,N/A
113,CRM,189.16,182.38,N/A


In [9]:
#remove glamour stocks
final_df.sort_values('Price-to-Earnings Ratio',ascending=False,inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio']>0]
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df.drop('index',axis=1,inplace=True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APTV,108.960,259.34,N/A
1,CRM,189.160,182.38,N/A
2,AMZN,143.280,128.08,N/A
3,ALB,280.430,125.79,N/A
4,ADSK,226.290,103.13,N/A
5,ARE,170.740,89.01,N/A
6,BSX,42.680,76.19,N/A
7,CDNS,195.290,70.07,N/A
8,CBOE,127.730,63.92,N/A
9,CMG,1711.310,62.76,N/A


In [10]:
#calculate number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')

portfolio_input()

Enter the size of your portfolio100000000


In [11]:
position_size = float(portfolio_size)/len(final_df)
for i in range(0,len(final_df)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i,'Price'])

final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,APTV,108.960,259.34,18355
1,CRM,189.160,182.38,10573
2,AMZN,143.280,128.08,13958
3,ALB,280.430,125.79,7131
4,ADSK,226.290,103.13,8838
5,ARE,170.740,89.01,11713
6,BSX,42.680,76.19,46860
7,CDNS,195.290,70.07,10241
8,CBOE,127.730,63.92,15658
9,CMG,1711.310,62.76,1168


In [12]:
# #more realistic value strategy
batch_api_call_url = f'https://sandbox.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#     Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
#     Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
#     Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
#     Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
#     Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [13]:
#robust value ->rv
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [16]:
rv_df = pd.DataFrame(columns = rv_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_df = rv_df.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.82,N/A,-4.99,N/A,-1.14,N/A,0.2267,N/A,110.683742,N/A,0.836918,N/A,N/A
1,AAP,214.55,N/A,23.23,N/A,4.42,N/A,1.19,N/A,13.191726,N/A,2.850302,N/A,N/A
2,AAPL,176.82,N/A,29.05,N/A,48.18,N/A,7.17,N/A,21.114323,N/A,16.333583,N/A,N/A
3,ABBV,147.28,N/A,20.47,N/A,17.37,N/A,4.54,N/A,11.335113,N/A,7.682539,N/A,N/A
4,ABC,154.08,N/A,17.59,N/A,144.36,N/A,0.139,N/A,9.528835,N/A,3.997570,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,7.50,N/A,44.35,N/A,1.73,N/A,1.26,N/A,19.955062,N/A,3.147238,N/A,N/A
111,CPB,52.41,N/A,16.87,N/A,4.66,N/A,1.84,N/A,16.596758,N/A,7.730127,N/A,N/A
112,CPRT,130.90,N/A,29.86,N/A,7,N/A,9.42,N/A,20.106922,N/A,10.574305,N/A,N/A
113,CRM,187.30,N/A,188.9,N/A,3.22,N/A,6.68,N/A,49.132603,N/A,9.014702,N/A,N/A


Missing data


In [17]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
27,ALXN,183.40,N/A,60.0,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
39,AON,301.58,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
70,BRK.B,299.90,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
87,CERN,97.02,N/A,48.67,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [18]:
rv_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [21]:
#replace missing data with avereage of the column of missing data
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(),inplace=True)
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [22]:
#Calculating Percentiles
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric],rv_df.loc[row,metric])

rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.82,N/A,-4.99,5.217391,-1.14,3.478261,0.2267,2.608696,110.683742,100.0,0.836918,5.217391,N/A
1,AAP,214.55,N/A,23.23,51.304348,4.42,53.043478,1.1900,16.521739,13.191726,38.26087,2.850302,19.130435,N/A
2,AAPL,176.82,N/A,29.05,62.608696,48.18,96.521739,7.1700,82.608696,21.114323,74.782609,16.333583,93.043478,N/A
3,ABBV,147.28,N/A,20.47,42.608696,17.37,88.695652,4.5400,68.695652,11.335113,30.434783,7.682539,50.434783,N/A
4,ABC,154.08,N/A,17.59,37.391304,144.36,99.130435,0.1390,1.73913,9.528835,21.73913,3.997570,25.217391,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,7.50,N/A,44.35,81.73913,1.73,17.391304,1.2600,18.695652,19.955062,68.695652,3.147238,21.73913,N/A
111,CPB,52.41,N/A,16.87,34.782609,4.66,55.652174,1.8400,31.73913,16.596758,54.782609,7.730127,52.173913,N/A
112,CPRT,130.90,N/A,29.86,64.347826,7.00,68.695652,9.4200,90.434783,20.106922,69.565217,10.574305,77.391304,N/A
113,CRM,187.30,N/A,188.90,99.130435,3.22,40.869565,6.6800,80.869565,49.132603,98.26087,9.014702,67.826087,N/A


In [26]:
#calculate RV Score
from statistics import mean
for row in rv_df.index:
    value_percentiles=[]
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row,'RV Score'] = mean(value_percentiles)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.82,N/A,-4.99,5.217391,-1.14,3.478261,0.2267,2.608696,110.683742,100.0,0.836918,5.217391,23.304348
1,AAP,214.55,N/A,23.23,51.304348,4.42,53.043478,1.1900,16.521739,13.191726,38.26087,2.850302,19.130435,35.652174
2,AAPL,176.82,N/A,29.05,62.608696,48.18,96.521739,7.1700,82.608696,21.114323,74.782609,16.333583,93.043478,81.913043
3,ABBV,147.28,N/A,20.47,42.608696,17.37,88.695652,4.5400,68.695652,11.335113,30.434783,7.682539,50.434783,56.173913
4,ABC,154.08,N/A,17.59,37.391304,144.36,99.130435,0.1390,1.73913,9.528835,21.73913,3.997570,25.217391,37.043478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,7.50,N/A,44.35,81.73913,1.73,17.391304,1.2600,18.695652,19.955062,68.695652,3.147238,21.73913,41.652174
111,CPB,52.41,N/A,16.87,34.782609,4.66,55.652174,1.8400,31.73913,16.596758,54.782609,7.730127,52.173913,45.826087
112,CPRT,130.90,N/A,29.86,64.347826,7.00,68.695652,9.4200,90.434783,20.106922,69.565217,10.574305,77.391304,74.086957
113,CRM,187.30,N/A,188.90,99.130435,3.22,40.869565,6.6800,80.869565,49.132603,98.26087,9.014702,67.826087,77.391304


In [27]:
rv_df.sort_values('RV Score',ascending=False,inplace=True)

In [31]:
rv_df = rv_df[:50]
rv_df.reset_index(drop=True,inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,CDNS,189.400,N/A,69.880000,93.913043,20.040000,90.434783,16.040000,100.0,48.567107,97.391304,17.185371,94.782609,95.304348
1,ADSK,233.620,N/A,104.390000,96.521739,71.360000,98.26087,10.940000,91.304348,61.471334,99.130435,11.731062,80.869565,93.217391
2,AMT,278.890,N/A,46.690000,85.217391,19.150000,89.565217,12.830000,98.26087,27.283909,87.826087,23.475434,99.130435,92.0
3,ADBE,440.486,N/A,42.300000,79.130435,14.710000,87.826087,12.110000,95.652174,32.036622,92.173913,13.668713,85.217391,88.0
4,ABMD,281.650,N/A,58.550000,90.434783,8.370000,74.782609,11.870000,94.782609,39.332941,96.521739,13.074421,81.73913,87.652174
5,CCI,184.050,N/A,51.200000,87.826087,9.950000,76.521739,11.610000,93.043478,25.143048,82.608696,21.363132,97.391304,87.478261
6,ANET,131.000,N/A,40.720000,77.391304,10.190000,78.26087,11.800000,93.913043,30.187151,90.434783,16.901649,93.913043,86.782609
7,ANSS,274.430,N/A,52.700000,88.695652,5.340000,60.434783,12.420000,96.521739,37.953139,93.913043,13.148461,82.608696,84.434783
8,BF.B,80.750,N/A,45.610000,83.478261,13.670000,86.956522,7.340000,83.478261,29.084169,88.695652,10.618631,78.26087,84.173913
9,ALB,282.880,N/A,123.560000,98.26087,5.300000,59.130435,7.700000,86.956522,23.019319,78.26087,23.344275,98.26087,84.173913


In [32]:
#Calculate number of shares to buy
portfolio_input()

Enter the size of your portfolio2500000


In [34]:
position_size = float(portfolio_size) / len(rv_df.index)

for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[row,'Price'])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,CDNS,189.400,263,69.880000,93.913043,20.040000,90.434783,16.040000,100.0,48.567107,97.391304,17.185371,94.782609,95.304348
1,ADSK,233.620,214,104.390000,96.521739,71.360000,98.26087,10.940000,91.304348,61.471334,99.130435,11.731062,80.869565,93.217391
2,AMT,278.890,179,46.690000,85.217391,19.150000,89.565217,12.830000,98.26087,27.283909,87.826087,23.475434,99.130435,92.0
3,ADBE,440.486,113,42.300000,79.130435,14.710000,87.826087,12.110000,95.652174,32.036622,92.173913,13.668713,85.217391,88.0
4,ABMD,281.650,177,58.550000,90.434783,8.370000,74.782609,11.870000,94.782609,39.332941,96.521739,13.074421,81.73913,87.652174
5,CCI,184.050,271,51.200000,87.826087,9.950000,76.521739,11.610000,93.043478,25.143048,82.608696,21.363132,97.391304,87.478261
6,ANET,131.000,381,40.720000,77.391304,10.190000,78.26087,11.800000,93.913043,30.187151,90.434783,16.901649,93.913043,86.782609
7,ANSS,274.430,182,52.700000,88.695652,5.340000,60.434783,12.420000,96.521739,37.953139,93.913043,13.148461,82.608696,84.434783
8,BF.B,80.750,619,45.610000,83.478261,13.670000,86.956522,7.340000,83.478261,29.084169,88.695652,10.618631,78.26087,84.173913
9,ALB,282.880,176,123.560000,98.26087,5.300000,59.130435,7.700000,86.956522,23.019319,78.26087,23.344275,98.26087,84.173913


In [35]:
writer = pd.ExcelWriter('value strategy.xlsx',engine = 'xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy')
writer.save()